In [2]:
%pylab widget
%load_ext autoreload
%autoreload 2
%load_ext memory_profiler

Populating the interactive namespace from numpy and matplotlib


In [3]:
import fastcat as fc
from glob import glob

## First I'll look at the detector MTF

### I'll load all of the kernels 

In [4]:
spectra = glob('../fastcat/data/MV_spectra/*.txt')
phantoms = ['Catphan_404','Catphan_515','Catphan_MTF']
MV_detectors = glob('../fastcat/data/Detectors/*')

dispatcher={'Catphan_404':fc.Catphan_404,
            'Catphan_515':fc.Catphan_515,
            'Catphan_MTF':fc.Catphan_MTF}

In [5]:
s = fc.Spectrum()

In [23]:
contrasts = []
mtfs = []
label = []

fig, axs = plt.subplots(3, 1)

spectra = [ 'kv',
 '../fastcat/data/MV_spectra/C_spectrum_25.txt',
 '../fastcat/data/MV_spectra/C_spectrum_6.txt',
 '../fastcat/data/MV_spectra/W_spectrum_25.txt',
 '../fastcat/data/MV_spectra/W_spectrum_6.txt']

MV_detectors = [
 '../fastcat/data/Detectors/CsI-784-micrometer',
 '../fastcat/data/Detectors/CWO-392-micrometer',
 '../fastcat/data/Detectors/CuGOS-784-micrometer']

phan_map = [
    'air',
    'G4_MUSCLE_SKELETAL_ICRP',
    'Spongiosa_Bone_ICRP',
    'G4_LUNG_MD_ICRP',
    'G4_BONE_COMPACT_ICRU',
    'G4_ADIPOSE_TISSUE_ICRP'
    ]
# s0 = fc.calculate_spectrum(60, 12, 3, 50)
# s0.attenuate(0.1,fc.get_mu(z=13))
# s0.attenuate(0.015,fc.get_mu(z=29))

s0 = fc.calculate_spectrum(120, 12, 3, 50)
s0.attenuate(0.3,fc.get_mu(z=13)) #was 0.3

s0.attenuate(0.089,fc.get_mu(z=22))
xx, yy = s0.get_points()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Calculation:  1 / 50
Calculation:  2 / 50
Calculation:  3 / 50
Calculation:  4 / 50
Calculation:  5 / 50
Calculation:  6 / 50
Calculation:  7 / 50
Calculation:  8 / 50
Calculation:  9 / 50
Calculation:  10 / 50
Calculation:  11 / 50
Calculation:  12 / 50
Calculation:  13 / 50
Calculation:  14 / 50
Calculation:  15 / 50
Calculation:  16 / 50
Calculation:  17 / 50
Calculation:  18 / 50
Calculation:  19 / 50
Calculation:  20 / 50
Calculation:  21 / 50
Calculation:  22 / 50
Calculation:  23 / 50
Calculation:  24 / 50
Calculation:  25 / 50
Calculation:  26 / 50
Calculation:  27 / 50
Calculation:  28 / 50
Calculation:  29 / 50
Calculation:  30 / 50
Calculation:  31 / 50
Calculation:  32 / 50
Calculation:  33 / 50
Calculation:  34 / 50
Calculation:  35 / 50
Calculation:  36 / 50
Calculation:  37 / 50
Calculation:  38 / 50
Calculation:  39 / 50
Calculation:  40 / 50
Calculation:  41 / 50
Calculation:  42 / 50
Calculation:  43 / 50
Calculation:  44 / 50
Calculation:  45 / 50
Calculation:  46 / 

In [7]:
from scipy.stats import poisson

In [8]:
noise_first.shape

NameError: name 'noise_first' is not defined

In [104]:
noise_temp = poisson.rvs(np.abs(phantom.proj)) - phantom.proj
plt.figure()
plt.plot(np.mean(noise_temp[5],0))

noise_first = np.random.poisson(np.abs(phantom.proj[0,10,:]),[200,phantom.proj.shape[2]])
noise_temp = noise_first[np.random.choice(noise_first.shape[0],phantom.proj.shape[:2])] - phantom.proj
plt.figure()
plt.plot(np.mean(noise_temp[5],0))

AttributeError: 'Catphan_404' object has no attribute 'proj'

In [25]:
nviews = 480

contrasts = []
mtfs = []
label = []
effs = []
nps = []
doses = []

for phan in phantoms[:1]:
    phantom = fc.Catphan_404(hi_res=False)
    phantom.phan_map = phan_map
    s = fc.Spectrum()
    for spectrum in spectra:
        for det in MV_detectors:#[1:2]:

            if len(spectrum) > 4:
                s.load(spectrum.split('/')[-1].split('.')[0])
                print('This is MV')
#                 s.y[-50:] = 0
#                 s.attenuate(0.2,fc.get_mu(z=13)) #was 0.3
#                 s.attenuate(0.089,fc.get_mu(z=22))
#                 s.attenuate(1,fc.get_mu(z=13))
            else:
                s.x = xx
                s.y = yy
                print('This is kV')
#                 s.attenuate(1,fc.get_mu(z=13))
                
            print(spectrum,det)
            kernel = fc.Detector(s, det.split('/')[-1].split('.')[0])
#             kernel.get_plot_mtf_real(axs[2])
#             mtfs.append([kernel.freq,kernel.mtf])
            angles = np.linspace(0,np.pi*2,nviews,endpoint=False)        
            if len(spectrum) < 4:
                phantom.return_projs(kernel,s,angles,det_on=True,convolve_on=False,mgy = 7/nviews,ASG=True,bowtie=True,filter='bowtie_mod',load_proj=True,proj_file='404_phan_480')
            else:
                phantom.return_projs(kernel,s,angles,det_on=True,mgy = 7/nviews,convolve_on=False,load_proj=True,proj_file='404_phan_480')
            phantom.reconstruct('FDK')
#             phantom.reconstruct('CGLS')
            contrasts.append(phantom.analyse_515(phantom.img[5],axs))
            label.append(spectrum.split('/')[-1].split('.')[0] + det.split('/')[-1].split('.')[0])
            effs.append(phantom.ratio)
            nps.append(phantom.nphoton_av)
            doses.append(phantom.doses)
            

#             break
#         break
#     break

[2021-07-26 12:20:40,933] {phantoms.py:649} INFO - Phantom is low resolution
This is kV
kv ../fastcat/data/Detectors/CsI-784-micrometer
[2021-07-26 12:20:40,942] {simulate.py:214} INFO - Initializing filter bowtie_mod
[2021-07-26 12:20:40,942] {simulate.py:219} INFO - Loading attenuations from 404_phan_480
[2021-07-26 12:20:40,961] {simulate.py:415} INFO - Initializing ASG
[2021-07-26 12:20:40,978] {simulate.py:534} INFO - Loading attenuations is on
[2021-07-26 12:20:40,979] {simulate.py:536} INFO - Running Simulations
[2021-07-26 12:20:40,979] {simulate.py:560} INFO -     Simulating 10 keV
[2021-07-26 12:20:41,686] {simulate.py:560} INFO -     Simulating 20 keV
[2021-07-26 12:20:42,467] {simulate.py:560} INFO -     Simulating 30 keV
[2021-07-26 12:20:43,286] {simulate.py:560} INFO -     Simulating 40 keV
[2021-07-26 12:20:44,437] {simulate.py:560} INFO -     Simulating 50 keV
[2021-07-26 12:20:45,700] {simulate.py:560} INFO -     Simulating 60 keV
[2021-07-26 12:20:47,016] {simulate.p

In [47]:
plt.figure()
for ii,eff in enumerate(zip(nps,effs)):
    plt.plot(ii,eff[0]/np.sum(nps),'r*',label='n_photons')
    plt.plot(ii,eff[1]/np.sum(effs),'b*',label='ratio')
plt.legend(['n_photon','ratio'])
# plt.legend(label)
# plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [56]:
phantom.weights_small2

array([0.0157925 , 0.02489176, 0.03072557, 0.03655938, 0.03827044,
       0.03381792, 0.0293654 , 0.0257994 , 0.02355599, 0.16430884,
       0.15787932, 0.0920026 , 0.06308159, 0.03618075, 0.08853884,
       0.10284713, 0.03638256, 0.        ])

In [57]:
plt.figure()
plt.plot(s.x,s.y)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [123]:
plt.figure()
plt.plot(np.array(doses[7]).T,'*')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
dep = np.load('../../fastcat/data/Detectors/CWO-392-micrometer/EnergyDeposition.npy')
MC_energies_keV = np.array(
    [
        10,
        20,
        30,
        40,
        50,
        60,
        70,
        80,
        90,
        100,
        300,
        500,
        700,
        900,
        1000,
        2000,
        4000,
        6000,
    ])

In [11]:
deps = []
deps.append(dep)
deps.append(MC_energies_keV)

In [13]:
# np.save('../../fastcat/data/Detectors/CWO-392-micrometer/EnergyDeposition.npy',deps)

In [26]:
inds_i_want = [0,3,2,1]
xx = np.linspace(0,1,5)#np.arange(5)

labs = ['120 kVp','2.5 MV C','6 MV C','2.5 MV W','6 MV W']
cc = np.array([np.array(contrasts[ii][3])[inds_i_want] for ii in range(len(contrasts))])
ee = np.array([np.array(contrasts[ii][4])[inds_i_want] for ii in range(len(contrasts))])
# ll = np.array([np.array(label[ii]) for ii in range(len(contrasts))])

inds = np.array([0,1,2,3,4])
# plt.figure(figsize=[8,4])#,dpi=300)
plt.figure(figsize=[10,5])#  ,dpi=300)

alph = 1
hat = None
plt.rcParams['axes.axisbelow'] = True

# plt.title('Contrast to Noise Ratio')
plt.subplot(131)
plt.title('CWO',{'fontsize':'x-large'})

x2,y2 = -0.5,31
plt.text(x2,y2,'(a)',color='cornflowerblue',fontsize=16)
plt.bar(xx-0.09,cc.flatten()[::4][1::3][inds], yerr = ee.flatten()[::4][1::3][inds],width=0.06,color='lightcoral',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.15,cc.flatten()[1::4][1::3][inds], yerr = ee.flatten()[1::4][1::3][inds],width=0.06,color='darkorange',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.21,cc.flatten()[3::4][1::3][inds], yerr = ee.flatten()[3::4][1::3][inds],width=0.06,color='cornflowerblue',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.27,cc.flatten()[2::4][1::3][inds], yerr = ee.flatten()[2::4][1::3][inds],width=0.06,color='k',alpha=alph,hatch=hat,fill=True)

plt.ylabel('CNR')
plt.xticks(np.array([-0.3,-0.05,0.2,0.45,0.70])+0.12,labs,rotation=45)
plt.grid('on')
plt.ylim([0,30])

off = 0
hat = None

plt.subplot(132)
plt.title('CsI',{'fontsize':'x-large'})
x2,y2 = -0.5,31
plt.text(x2,y2,'(b)',color='cornflowerblue',fontsize=16)
plt.grid('on')
plt.bar(xx-0.27+off,cc.flatten()[2::4][::3][inds], yerr = ee.flatten()[2::4][::3][inds],width=0.06,color='k',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.21+off,cc.flatten()[3::4][::3][inds], yerr = ee.flatten()[3::4][::3][inds],width=0.06,color='cornflowerblue',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.15+off,cc.flatten()[1::4][::3][inds], yerr = ee.flatten()[1::4][::3][inds],width=0.06,color='darkorange',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.09+off,cc.flatten()[::4][::3][inds], yerr = ee.flatten()[::4][::3][inds],width=0.06,color='lightcoral',alpha=alph,hatch=hat,fill=True)
plt.ylim([0,30])
plt.yticks([0,5,10,15,20,25,30],labels=[])
plt.xticks(np.array([-0.3,-0.05,0.2,0.45,0.70])+0.12,labs,rotation=45)
off = 0
plt.subplot(133)
plt.title('GOS',{'fontsize':'x-large'})
x2,y2 = -0.5,31
plt.text(x2,y2,'(c)',color='cornflowerblue',fontsize=16)
plt.grid('on')
plt.bar(xx-0.27+off,cc.flatten()[2::4][2::3][inds], yerr = ee.flatten()[2::4][2::3][inds],width=0.06,color='k',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.21+off,cc.flatten()[3::4][2::3][inds], yerr = ee.flatten()[3::4][2::3][inds],width=0.06,color='cornflowerblue',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.15+off,cc.flatten()[1::4][2::3][inds], yerr = ee.flatten()[1::4][2::3][inds],width=0.06,color='darkorange',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.09+off,cc.flatten()[::4][2::3][inds], yerr = ee.flatten()[::4][2::3][inds],width=0.06,color='lightcoral',alpha=alph,hatch=hat,fill=True)

plt.ylim([0,30])
plt.yticks([0,5,10,15,20,25,30],labels=[])
plt.xticks(np.array([-0.3,-0.05,0.2,0.45,0.70])+0.12,labs,rotation=45)

plt.legend(['Rib Bone','Lung','Adipose','Spongiosa'])
# ax = plt.gca().add_artist(first_legend)


plt.tight_layout()
# plt.savefig('/home/xcite/fastcat_paper_2_tmi/figures/CNR.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [66]:
inds_i_want = [0,3,2,1]
xx = np.linspace(0,1,5)#np.arange(5)

labs = ['120 kVp','2.5 MV C','6 MV C','2.5 MV W','6 MV W']
cc = np.array([np.array(contrasts[ii][1])[inds_i_want] for ii in range(len(contrasts))])
ee = np.array([np.array(contrasts[ii][2])[inds_i_want] for ii in range(len(contrasts))])
# ll = np.array([np.array(label[ii]) for ii in range(len(contrasts))])

inds = np.array([0,1,2,3,4])
# plt.figure(figsize=[8,4])#,dpi=300)
plt.figure(figsize=[10,5])#  ,dpi=300)

alph = 1
hat = None
plt.rcParams['axes.axisbelow'] = True

# plt.title('Contrast to Noise Ratio')
plt.subplot(131)
plt.title('CWO',{'fontsize':'x-large'})

x2,y2 = -0.5,31
plt.text(x2,y2,'(a)',color='cornflowerblue',fontsize=16)
plt.bar(xx-0.09,cc.flatten()[::4][1::3][inds], yerr = ee.flatten()[::4][1::3][inds],width=0.06,color='lightcoral',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.15,cc.flatten()[1::4][1::3][inds], yerr = ee.flatten()[1::4][1::3][inds],width=0.06,color='darkorange',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.21,cc.flatten()[3::4][1::3][inds], yerr = ee.flatten()[3::4][1::3][inds],width=0.06,color='cornflowerblue',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.27,cc.flatten()[2::4][1::3][inds], yerr = ee.flatten()[2::4][1::3][inds],width=0.06,color='k',alpha=alph,hatch=hat,fill=True)

plt.ylabel('CNR')
plt.xticks(np.array([-0.3,-0.05,0.2,0.45,0.70])+0.12,labs,rotation=45)
plt.grid('on')
plt.ylim([0,90])

off = 0
hat = None

plt.subplot(132)
plt.title('CsI',{'fontsize':'x-large'})
x2,y2 = -0.5,31
plt.text(x2,y2,'(b)',color='cornflowerblue',fontsize=16)
plt.grid('on')
plt.bar(xx-0.27+off,cc.flatten()[2::4][::3][inds], yerr = ee.flatten()[2::4][::3][inds],width=0.06,color='k',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.21+off,cc.flatten()[3::4][::3][inds], yerr = ee.flatten()[3::4][::3][inds],width=0.06,color='cornflowerblue',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.15+off,cc.flatten()[1::4][::3][inds], yerr = ee.flatten()[1::4][::3][inds],width=0.06,color='darkorange',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.09+off,cc.flatten()[::4][::3][inds], yerr = ee.flatten()[::4][::3][inds],width=0.06,color='lightcoral',alpha=alph,hatch=hat,fill=True)
plt.ylim([0,90])
# plt.yticks([0,5,10,15,20,25,30],labels=[])
plt.xticks(np.array([-0.3,-0.05,0.2,0.45,0.70])+0.12,labs,rotation=45)
off = 0
plt.subplot(133)
plt.title('GOS',{'fontsize':'x-large'})
x2,y2 = -0.5,31
plt.text(x2,y2,'(c)',color='cornflowerblue',fontsize=16)
plt.grid('on')
plt.bar(xx-0.27+off,cc.flatten()[2::4][2::3][inds], yerr = ee.flatten()[2::4][2::3][inds],width=0.06,color='k',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.21+off,cc.flatten()[3::4][2::3][inds], yerr = ee.flatten()[3::4][2::3][inds],width=0.06,color='cornflowerblue',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.15+off,cc.flatten()[1::4][2::3][inds], yerr = ee.flatten()[1::4][2::3][inds],width=0.06,color='darkorange',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.09+off,cc.flatten()[::4][2::3][inds], yerr = ee.flatten()[::4][2::3][inds],width=0.06,color='lightcoral',alpha=alph,hatch=hat,fill=True)

plt.ylim([0,90])
# plt.yticks([0,5,10,15,20,25,30],labels=[])
plt.xticks(np.array([-0.3,-0.05,0.2,0.45,0.70])+0.12,labs,rotation=45)

plt.legend(['Rib Bone','Lung','Adipose','Spongiosa'])
# ax = plt.gca().add_artist(first_legend)


plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [129]:
plt.figure()
plt.imshow(phantom.img[5])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [146]:
# inds_i_want = [0,6,12,18,21,24]
inds_i_want = [0,3,2,1]
xx = np.linspace(0,1,5)#np.arange(5)

labs = ['120 kVp','2.5 MV C','6 MV C','2.5 MV W','6 MV W']
cc = np.array([np.array(contrasts[ii][3])[inds_i_want] for ii in range(len(contrasts))])
ee = np.array([np.array(contrasts[ii][4])[inds_i_want] for ii in range(len(contrasts))])
# ll = np.array([np.array(label[ii]) for ii in range(len(contrasts))])

inds = np.array([0,1,2,3,4])
plt.figure(figsize=[8,4])#,dpi=300)
# plt.figure(figsize=[10,5]),dpi=300)

alph = 1
hat = None
plt.rcParams['axes.axisbelow'] = True

plt.title('CWO',{'fontsize':'x-large'})

x2,y2 = -0.5,31
plt.text(x2,y2,'(a)',color='cornflowerblue',fontsize=16)
plt.bar(xx-0.09,cc.flatten()[::4][1::3][inds], yerr = ee.flatten()[::4][1::3][inds],width=0.06,color='lightcoral',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.15,cc.flatten()[1::4][1::3][inds], yerr = ee.flatten()[1::4][1::3][inds],width=0.06,color='darkorange',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.21,cc.flatten()[3::4][1::3][inds], yerr = ee.flatten()[3::4][1::3][inds],width=0.06,color='cornflowerblue',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.27,cc.flatten()[2::4][1::3][inds], yerr = ee.flatten()[2::4][1::3][inds],width=0.06,color='k',alpha=alph,hatch=hat,fill=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<BarContainer object of 5 artists>

## Installation

Dear entrepid hypothetical users,

There are two install options: The second is not to install Fastcat and use it on google colaboratory. So keep that in mind and do it if your installation fails on your local computer.

### Dependencies

To run fastcat locally you need:

* CUDA Capable GPU
* Nvidia CUDA toolkit
* TIGRE
* numpy, scipy, matplotlib
* Some other stuff you can install with pip/conda

If you don't have a scientific python installation? Go away! I'm really not the person to explain how to do that. Somewhere else explains it better somewhere else. Just google it and come back when you are done

#### TIGRE

To run Fastcat you need a python package called TIGRE. If you can install TIGRE then you can use Fastcat without issue.

So I'm going to direct you to the TIGRE installation for python. The crux move is really the CUDA installation which sometimes works using conda but other times does not.

In [14]:
np.save('cnr_nonoise_404',contrasts)
np.save('mtfs_nonoise_404',mtfs)
np.save('labels_404',label)

In [1]:
%pylab widget

Populating the interactive namespace from numpy and matplotlib


In [3]:
contrasts = np.load('../data_second_paper/cnr_new_noisy_404.npy',allow_pickle=True)
mtfs = np.load('../data_second_paper/mtfs_nonoise_404.npy')
label = np.load('../data_second_paper/labels_404.npy')

# This is the good figure I think

In [89]:
# inds_i_want = [0,6,12,18,21,24]
inds_i_want = [0,3,2,1]
xx = np.linspace(0,1,5)#np.arange(5)

labs = ['120 kVp','2.5 MV C','6 MV C','2.5 MV W','6 MV W']
cc = np.array([np.array(contrasts[ii][3])[inds_i_want] for ii in range(len(contrasts))])
ee = np.array([np.array(contrasts[ii][4])[inds_i_want] for ii in range(len(contrasts))])
# ll = np.array([np.array(label[ii]) for ii in range(len(contrasts))])

inds = np.array([0,1,2,3,4])
# plt.figure(figsize=[8,4])#,dpi=300)
plt.figure(figsize=[10,5],dpi=300)

alph = 1
hat = None
plt.rcParams['axes.axisbelow'] = True

# plt.title('Contrast to Noise Ratio')
plt.subplot(131)
plt.title('CWO',{'fontsize':'x-large'})

x2,y2 = -0.5,31
plt.text(x2,y2,'(a)',color='cornflowerblue',fontsize=16)
plt.bar(xx-0.09,cc.flatten()[::4][1::3][inds], yerr = ee.flatten()[::4][1::3][inds],width=0.06,color='lightcoral',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.15,cc.flatten()[1::4][1::3][inds], yerr = ee.flatten()[1::4][1::3][inds],width=0.06,color='darkorange',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.21,cc.flatten()[3::4][1::3][inds], yerr = ee.flatten()[3::4][1::3][inds],width=0.06,color='cornflowerblue',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.27,cc.flatten()[2::4][1::3][inds], yerr = ee.flatten()[2::4][1::3][inds],width=0.06,color='k',alpha=alph,hatch=hat,fill=True)

plt.ylabel('CNR')
plt.xticks(np.array([-0.3,-0.05,0.2,0.45,0.70])+0.12,labs,rotation=45)
# plt.bar(xx-0.09*3,cc.flatten()[2::4][1::3][inds], yerr = ee.flatten()[2::4][1::3][inds],width=0.06,color='cornflowerblue',alpha=alph,hatch=hat,fill=True)
# plt.bar(xx-0.03*3,cc.flatten()[3::4][1::3][inds], yerr = ee.flatten()[3::4][1::3][inds],width=0.06,color='k',alpha=alph - 0.5,hatch=hat,fill=True)
# plt.bar(xx-0.03*3,cc.flatten()[2::4][::3][inds], yerr = ee.flatten()[2::4][::3][inds],width=0.06,color='cornflowerblue',alpha=alph,hatch=hat,fill=True)
# plt.bar(xx+0.03*3,cc.flatten()[2::4][2::3][inds], yerr = ee.flatten()[2::4][2::3][inds],width=0.06,color='cornflowerblue',alpha=alph,hatch=hat,fill=True)
plt.grid('on')
plt.ylim([0,30])
# plt.bar(xx+0.09*3,cc.flatten()[::4][1::3][inds], yerr = ee.flatten()[::4][1::3][inds],width=0.06,color='lightcoral',alpha=alph,hatch=hat,fill=True)

off = 0
# alph = 0.85
hat = None

plt.subplot(132)
plt.title('CsI',{'fontsize':'x-large'})
x2,y2 = -0.5,31
plt.text(x2,y2,'(b)',color='cornflowerblue',fontsize=16)
# xx = xx + 1.6
plt.grid('on')
plt.bar(xx-0.27+off,cc.flatten()[2::4][::3][inds], yerr = ee.flatten()[2::4][::3][inds],width=0.06,color='k',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.21+off,cc.flatten()[3::4][::3][inds], yerr = ee.flatten()[3::4][::3][inds],width=0.06,color='cornflowerblue',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.15+off,cc.flatten()[1::4][::3][inds], yerr = ee.flatten()[1::4][::3][inds],width=0.06,color='darkorange',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.09+off,cc.flatten()[::4][::3][inds], yerr = ee.flatten()[::4][::3][inds],width=0.06,color='lightcoral',alpha=alph,hatch=hat,fill=True)
plt.ylim([0,30])
plt.yticks([0,5,10,15,20,25,30],labels=[])
plt.xticks(np.array([-0.3,-0.05,0.2,0.45,0.70])+0.12,labs,rotation=45)
off = 0
# alph = 0.7

# xx += 1.6
# hat = '\\\\\\\\\\\\\\\\\\'
plt.subplot(133)
plt.title('GOS',{'fontsize':'x-large'})
x2,y2 = -0.5,31
plt.text(x2,y2,'(c)',color='cornflowerblue',fontsize=16)
plt.grid('on')
plt.bar(xx-0.27+off,cc.flatten()[2::4][2::3][inds], yerr = ee.flatten()[2::4][2::3][inds],width=0.06,color='k',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.21+off,cc.flatten()[3::4][2::3][inds], yerr = ee.flatten()[3::4][2::3][inds],width=0.06,color='cornflowerblue',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.15+off,cc.flatten()[1::4][2::3][inds], yerr = ee.flatten()[1::4][2::3][inds],width=0.06,color='darkorange',alpha=alph,hatch=hat,fill=True)
plt.bar(xx-0.09+off,cc.flatten()[::4][2::3][inds], yerr = ee.flatten()[::4][2::3][inds],width=0.06,color='lightcoral',alpha=alph,hatch=hat,fill=True)

plt.ylim([0,30])
plt.yticks([0,5,10,15,20,25,30],labels=[])
plt.xticks(np.array([-0.3,-0.05,0.2,0.45,0.70])+0.12,labs,rotation=45)
# plt.ylabel('CNR')
# labs = ['2.5 MV C','120 kVp','6 MV C','2.5 MV W','6 MV W']
# plt.xticks(range(len(label[::3])),labels=labs,rotation=45)

# from matplotlib.lines import Line2D

# custom_lines = [Line2D([0], [0],marker='s',color = 'w', markerfacecolor='k', lw=4),
#                 Line2D([0], [0],marker='s',color = 'w', markerfacecolor='cornflowerblue', lw=4),
#                 Line2D([0], [0],marker='s',color = 'w', markerfacecolor='darkorange', lw=4),
#                 Line2D([0], [0],marker='s',color = 'w', markerfacecolor='lightcoral', lw=4)]

# # fig, ax = plt.subplots()
# # lines = ax.plot(data)
plt.legend(['Rib Bone','Lung','Adipose','Spongiosa'])
# plt.grid('on')
# Create a legend for the first line.
# first_legend = plt.legend(handles=[line1])

# Add the legend manually to the current Axes.
# ax = plt.gca().add_artist(first_legend)


plt.tight_layout()
# plt.savefig('/home/xcite/fastcat_paper_2_tmi/figures/CNR.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# This is a new analysis on the weights

In [14]:
spectra = glob('../fastcat/data/MV_spectra/*.txt')
phantoms = ['Catphan_404','Catphan_515','Catphan_MTF']
MV_detectors = glob('../fastcat/data/Detectors/*')

dispatcher={'Catphan_404':fc.Catphan_404,
            'Catphan_515':fc.Catphan_515,
            'Catphan_MTF':fc.Catphan_MTF}

In [15]:
s = fc.Spectrum()

In [62]:
contrasts = []
mtfs = []
label = []

fig, axs = plt.subplots(3, 1)

spectra = [ 'kv',
 '../fastcat/data/MV_spectra/C_spectrum_25.txt',
 '../fastcat/data/MV_spectra/C_spectrum_6.txt',
 '../fastcat/data/MV_spectra/W_spectrum_25.txt',
 '../fastcat/data/MV_spectra/W_spectrum_6.txt']

MV_detectors = [
 '../fastcat/data/Detectors/CsI-784-micro_thin',
 '../fastcat/data/Detectors/CWO-784-micrometer',
 '../fastcat/data/Detectors/CuGOS-784-micrometer']

phan_map = [
    'air',
    'G4_MUSCLE_SKELETAL_ICRP',
    'Spongiosa_Bone_ICRP',
    'G4_LUNG_MD_ICRP',
    'G4_BONE_COMPACT_ICRU',
    'G4_ADIPOSE_TISSUE_ICRP'
    ]
# s0 = fc.calculate_spectrum(60, 12, 3, 50)
# s0.attenuate(0.1,fc.get_mu(z=13))
# s0.attenuate(0.015,fc.get_mu(z=29))

s0 = fc.calculate_spectrum(120, 12, 3, 50)
s0.attenuate(0.2,fc.get_mu(z=13)) #was 0.3
# s0.attenuate(0.089,fc.get_mu(z=22))
xx, yy = s0.get_points()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Calculation:  1 / 50
Calculation:  2 / 50
Calculation:  3 / 50
Calculation:  4 / 50
Calculation:  5 / 50
Calculation:  6 / 50
Calculation:  7 / 50
Calculation:  8 / 50
Calculation:  9 / 50
Calculation:  10 / 50
Calculation:  11 / 50
Calculation:  12 / 50
Calculation:  13 / 50
Calculation:  14 / 50
Calculation:  15 / 50
Calculation:  16 / 50
Calculation:  17 / 50
Calculation:  18 / 50
Calculation:  19 / 50
Calculation:  20 / 50
Calculation:  21 / 50
Calculation:  22 / 50
Calculation:  23 / 50
Calculation:  24 / 50
Calculation:  25 / 50
Calculation:  26 / 50
Calculation:  27 / 50
Calculation:  28 / 50
Calculation:  29 / 50
Calculation:  30 / 50
Calculation:  31 / 50
Calculation:  32 / 50
Calculation:  33 / 50
Calculation:  34 / 50
Calculation:  35 / 50
Calculation:  36 / 50
Calculation:  37 / 50
Calculation:  38 / 50
Calculation:  39 / 50
Calculation:  40 / 50
Calculation:  41 / 50
Calculation:  42 / 50
Calculation:  43 / 50
Calculation:  44 / 50
Calculation:  45 / 50
Calculation:  46 / 

In [64]:
nviews = 2

weights_1 = []
weights_2 = []
photons = []

for phan in phantoms[:1]:
    phantom = fc.Catphan_404(hi_res=False)
    phantom.phan_map = phan_map
    s = fc.Spectrum()
    for spectrum in spectra:#[:2]:
        for det in MV_detectors:#[1:2]:

            if len(spectrum) > 4:
                s.load(spectrum.split('/')[-1].split('.')[0])
                print('This is MV')
#                 s.attenuate(1,fc.get_mu(z=13))
            else:
                s.x = xx
                s.y = yy
                print('This is kV')
#                 s.attenuate(1,fc.get_mu(z=13))
            
            print(spectrum,det)
            plt.figure()
            plt.plot(s.x,s.y)
            kernel = fc.Detector(s, det.split('/')[-1].split('.')[0])
            kernel.get_plot_mtf_real(axs[2])
            mtfs.append([kernel.freq,kernel.mtf])
            angles = np.linspace(0,np.pi*2,nviews)        
#             if len(spectrum) < 4:
#                 phantom.return_projs(kernel,s,angles,det_on=True,mgy = 7/nviews,ASG=True,bowtie=True,filter='bowtie_mod')
#             else:
            phantom.return_projs(kernel,s,angles,det_on=True,mgy = 7/180)
            weights_1.append(phantom.weights_small)
            weights_2.append(phantom.weights_small2)
            
            photons.append([phantom.nphoton_av,phantom.ratio])
            
            
            label.append(spectrum.split('/')[-1].split('.')[0] + det.split('/')[-1].split('.')[0])

#             break
#         break
#     break

[2021-07-14 19:29:51,660] {phantoms.py:649} INFO - Phantom is low resolution
This is kV
kv ../fastcat/data/Detectors/CsI-784-micro_thin


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[2021-07-14 19:29:51,866] {legend.py:1193} WARNING - No handles with labels found to put in legend.
[2021-07-14 19:29:51,922] {simulate.py:469} INFO - Running Simulations
[2021-07-14 19:29:51,922] {simulate.py:484} INFO -     Simulating 10 keV
[2021-07-14 19:29:52,083] {simulate.py:484} INFO -     Simulating 20 keV
[2021-07-14 19:29:52,211] {simulate.py:484} INFO -     Simulating 30 keV
[2021-07-14 19:29:52,335] {simulate.py:484} INFO -     Simulating 40 keV
[2021-07-14 19:29:52,462] {simulate.py:484} INFO -     Simulating 50 keV
[2021-07-14 19:29:52,587] {simulate.py:484} INFO -     Simulating 60 keV
[2021-07-14 19:29:52,715] {simulate.py:484} INFO -     Simulating 70 keV
[2021-07-14 19:29:52,841] {simulate.py:484} INFO -     Simulating 80 keV
[2021-07-14 19:29:52,965] {simulate.py:484} INFO -     Simulating 90 keV
[2021-07-14 19:29:53,091] {simulate.py:484} INFO -     Simulating 100 keV
[2021-07-14 19:29:53,215] {simulate.py:619} INFO - Weighting simulations
[2021-07-14 19:29:53,216]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[2021-07-14 19:29:53,416] {legend.py:1193} WARNING - No handles with labels found to put in legend.
[2021-07-14 19:29:53,419] {simulate.py:248} INFO - This is a small edep file starting at 30
[2021-07-14 19:29:53,467] {simulate.py:469} INFO - Running Simulations
[2021-07-14 19:29:53,468] {simulate.py:484} INFO -     Simulating 30 keV
[2021-07-14 19:29:53,591] {simulate.py:484} INFO -     Simulating 40 keV
[2021-07-14 19:29:53,718] {simulate.py:484} INFO -     Simulating 50 keV
[2021-07-14 19:29:53,843] {simulate.py:484} INFO -     Simulating 60 keV
[2021-07-14 19:29:53,971] {simulate.py:484} INFO -     Simulating 70 keV
[2021-07-14 19:29:54,099] {simulate.py:484} INFO -     Simulating 80 keV
[2021-07-14 19:29:54,223] {simulate.py:484} INFO -     Simulating 90 keV
[2021-07-14 19:29:54,353] {simulate.py:484} INFO -     Simulating 100 keV
[2021-07-14 19:29:54,480] {simulate.py:619} INFO - Weighting simulations
[2021-07-14 19:29:54,480] {simulate.py:686} INFO -     Added noise 596.54266300

...

[2021-07-14 19:30:16,274] {legend.py:1193} WARNING - No handles with labels found to put in legend.
[2021-07-14 19:30:16,327] {simulate.py:469} INFO - Running Simulations
[2021-07-14 19:30:16,327] {simulate.py:484} INFO -     Simulating 10 keV
[2021-07-14 19:30:16,455] {simulate.py:484} INFO -     Simulating 20 keV
[2021-07-14 19:30:16,578] {simulate.py:484} INFO -     Simulating 30 keV
[2021-07-14 19:30:16,704] {simulate.py:484} INFO -     Simulating 40 keV
[2021-07-14 19:30:16,830] {simulate.py:484} INFO -     Simulating 50 keV
[2021-07-14 19:30:16,951] {simulate.py:484} INFO -     Simulating 60 keV
[2021-07-14 19:30:17,076] {simulate.py:484} INFO -     Simulating 70 keV
[2021-07-14 19:30:17,206] {simulate.py:484} INFO -     Simulating 80 keV
[2021-07-14 19:30:17,332] {simulate.py:484} INFO -     Simulating 90 keV
[2021-07-14 19:30:17,459] {simulate.py:484} INFO -     Simulating 100 keV
[2021-07-14 19:30:17,585] {simulate.py:484} INFO -     Simulating 300 keV
[2021-07-14 19:30:17,717

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[2021-07-14 19:30:18,822] {legend.py:1193} WARNING - No handles with labels found to put in legend.
[2021-07-14 19:30:18,825] {simulate.py:248} INFO - This is a small edep file starting at 30
[2021-07-14 19:30:18,874] {simulate.py:469} INFO - Running Simulations
[2021-07-14 19:30:18,875] {simulate.py:484} INFO -     Simulating 30 keV
[2021-07-14 19:30:19,002] {simulate.py:484} INFO -     Simulating 40 keV
[2021-07-14 19:30:19,129] {simulate.py:484} INFO -     Simulating 50 keV
[2021-07-14 19:30:19,255] {simulate.py:484} INFO -     Simulating 60 keV
[2021-07-14 19:30:19,380] {simulate.py:484} INFO -     Simulating 70 keV
[2021-07-14 19:30:19,506] {simulate.py:484} INFO -     Simulating 80 keV
[2021-07-14 19:30:19,629] {simulate.py:484} INFO -     Simulating 90 keV
[2021-07-14 19:30:19,751] {simulate.py:484} INFO -     Simulating 100 keV
[2021-07-14 19:30:19,874] {simulate.py:484} INFO -     Simulating 300 keV
[2021-07-14 19:30:20,003] {simulate.py:484} INFO -     Simulating 500 keV
[202

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[2021-07-14 19:30:20,999] {legend.py:1193} WARNING - No handles with labels found to put in legend.
[2021-07-14 19:30:21,052] {simulate.py:469} INFO - Running Simulations
[2021-07-14 19:30:21,052] {simulate.py:484} INFO -     Simulating 20 keV
[2021-07-14 19:30:21,180] {simulate.py:484} INFO -     Simulating 30 keV
[2021-07-14 19:30:21,309] {simulate.py:484} INFO -     Simulating 40 keV
[2021-07-14 19:30:21,434] {simulate.py:484} INFO -     Simulating 50 keV
[2021-07-14 19:30:21,557] {simulate.py:484} INFO -     Simulating 60 keV
[2021-07-14 19:30:21,683] {simulate.py:484} INFO -     Simulating 70 keV
[2021-07-14 19:30:21,805] {simulate.py:484} INFO -     Simulating 80 keV
[2021-07-14 19:30:21,932] {simulate.py:484} INFO -     Simulating 90 keV
[2021-07-14 19:30:22,058] {simulate.py:484} INFO -     Simulating 100 keV
[2021-07-14 19:30:22,182] {simulate.py:484} INFO -     Simulating 300 keV
[2021-07-14 19:30:22,310] {simulate.py:484} INFO -     Simulating 500 keV
[2021-07-14 19:30:22,44

In [65]:
plt.figure(figsize=[10,15])

for ii,zipper in enumerate(zip(weights_1,weights_2,label)):
    if zipper[1].shape[0] == 18:
        original_energies_keV = np.array(
            [
                10,
                20,
                30,
                40,
                50,
                60,
                70,
                80,
                90,
                100,
                300,
                500,
                700,
                900,
                1000,
                2000,
                4000,
                6000,
            ]
        )
    else:
        original_energies_keV = np.array(
            [
                30,
                40,
                50,
                60,
                70,
                80,
                90,
                100,
                300,
                500,
                700,
                900,
                1000,
                2000,
                4000,
                6000,
            ]
        )

    plt.subplot(5,3,ii+1)
#     plt.plot(zipper[0])
    plt.plot(original_energies_keV,zipper[0])#/original_energies_keV*10)
    print(np.nansum(zipper[1]))
    plt.title(zipper[2].split('-')[0])
    plt.xlabel(zipper[0])
#     plt.text(0,0,photons[ii][:])
    print(zipper[2].split('-')[0],photons[ii])
#     plt.ylim([0,1])
    
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1.0
kvCsI [0.27276342216586136, 662.5759780627903]
1.0
kvCWO [0.8982738182577514, 664.0989093562347]
1.0
kvCuGOS [0.07488838934468509, 662.6469578737214]
1.0
C_spectrum_25CsI [0.07879379902987424, 157.96481137635655]
0.9999999999999999
C_spectrum_25CWO [0.7020465059778731, 157.96704952116028]
1.0
C_spectrum_25CuGOS [0.0469026923467256, 157.96484307044722]
1.0
C_spectrum_6CsI [0.049724374288624855, 73.4583136504904]
1.0
C_spectrum_6CWO [0.5622261778516847, 75.87803400196803]
1.0
C_spectrum_6CuGOS [0.03107246778754077, 73.45831450423182]
1.0
W_spectrum_25CsI [0.005732800527089785, 80.46170547840256]
1.0
W_spectrum_25CWO [0.48373473124479643, 80.46170554302955]
1.0
W_spectrum_25CuGOS [0.009604853453948143, 80.46170547958695]
1.0
W_spectrum_6CsI [0.0022025355054545548, 41.48363690412265]
1.0
W_spectrum_6CWO [0.33230427801628526, 43.608436411882096]
0.9999999999999999
W_spectrum_6CuGOS [0.00504234156065385, 41.48363733414015]
